In [1]:
from buildstock_query import BuildStockQuery, ExeId, MappedColumn
import pandas as pd
import sqlalchemy as sa

In [2]:
my_run = BuildStockQuery(db_name='euss-tests',
                        table_name='res_test_03_2018_10k_20220607',
                        workgroup='eulp',
                        buildstock_type='resstock',
                        skip_reports=True)

INFO:buildstock_query.query_core:Loading res_test_03_2018_10k_20220607 ...
INFO:buildstock_query.query_core:14 queries cache read from res_test_03_2018_10k_20220607_query_cache.pkl.
INFO:buildstock_query.query_core:14 queries cache is updated.


In [3]:
query = my_run.get_building_ids(get_query_only=True)
query

'SELECT res_test_03_2018_10k_20220607_baseline.building_id \nFROM res_test_03_2018_10k_20220607_baseline'

In [4]:
building_type_map = {"Mobile Home": "MH", "Single-Family Detached": "SF",
                     "Single-Family Attached": "SF", "Multi-Family with 2 - 4 Units": "MF",
                     "Multi-Family with 5+ Units": "MF",}
bldg_col = my_run.get_column('build_existing_model.geometry_building_type_recs')
simple_bldg_col = MappedColumn(bsq=my_run, name='simple_bldg_type', mapping_dict=building_type_map,
                               key=bldg_col)
mapped_agg_query = my_run.agg.aggregate_annual(enduses=['fuel_use_electricity_total_m_btu'],
                                       group_by=[simple_bldg_col],
                                          get_query_only=False)
print(mapped_agg_query)

  simple_bldg_type  sample_count   units_count  \
0               MF          1635  2.177363e+07   
1               MH           703  9.361996e+06   
2               SF          7651  1.018899e+08   

   fuel_use_electricity_total_m_btu  
0                      5.484767e+08  
1                      4.506122e+08  
2                      4.623009e+09  


In [7]:
dryer_co2_impact = {"Gas, 120% Usage": 1.2 * 1.5, 
                    "Gas, 100% Usage": 1 * 1.5,
                    "Gas, 80% Usage": 0.8 * 1.5,
                    "Electric, 120% Usage": 1.2 * 0.5,
                    "Electric 100% Usage": 1 * 0.5,
                    "Electric 80% Usage": 0.8 * 0.5,
                    "None": 0}
bldg_col = my_run.get_column('build_existing_model.clothes_dryer')
impact_col = MappedColumn(bsq=my_run, name='dryer_impact', mapping_dict=dryer_co2_impact,
                               key=bldg_col)
mapped_agg_query = my_run.agg.aggregate_annual(enduses=[impact_col],
                                               get_query_only=False)
print(mapped_agg_query)

   sample_count   units_count  dryer_impact
0          9989  1.330256e+08  5.397863e+07


In [10]:
baseline_agg_query = my_run.agg.aggregate_annual(enduses=['fuel_use_electricity_total_m_btu'],
                                       group_by=['geometry_building_type_recs'],
                                          get_query_only=True,
                                          )
print(baseline_agg_query)



SELECT res_test_03_2018_10k_20220607_baseline."build_existing_model.geometry_building_type_recs" AS geometry_building_type_recs, sum(1) AS sample_count, sum(res_test_03_2018_10k_20220607_baseline."build_existing_model.sample_weight") AS units_count, sum(res_test_03_2018_10k_20220607_baseline."report_simulation_output.fuel_use_electricity_total_m_btu" * res_test_03_2018_10k_20220607_baseline."build_existing_model.sample_weight") AS fuel_use_electricity_total_m_btu 
FROM res_test_03_2018_10k_20220607_baseline 
WHERE res_test_03_2018_10k_20220607_baseline.completed_status = 'Success' GROUP BY 1 ORDER BY 1


In [11]:
baseline_agg = my_run.execute("""
SELECT "res_test_03_2018_10k_20220607_baseline"."build_existing_model.geometry_building_type_recs" AS "geometry_building_type_recs", sum(1) AS "sample_count", sum("res_test_03_2018_10k_20220607_baseline"."build_existing_model.sample_weight") AS "units_count", sum("res_test_03_2018_10k_20220607_baseline"."report_simulation_output.fuel_use_electricity_total_m_btu" * "res_test_03_2018_10k_20220607_baseline"."build_existing_model.sample_weight") AS "fuel_use_electricity_total_m_btu" 
FROM "res_test_03_2018_10k_20220607_baseline" 
WHERE "res_test_03_2018_10k_20220607_baseline"."completed_status" = 'Success' GROUP BY 1
""")
baseline_agg

,geometry_building_type_recs,sample_count,units_count,fuel_use_electricity_total_m_btu
0,Multi-Family with 5+ Units,1189,1.583416e+07,4.099084e+08
1,Single-Family Attached,530,7.058119e+06,2.094629e+08
2,Single-Family Detached,7121,9.483182e+07,4.413546e+09
3,Mobile Home,703,9.361996e+06,4.506122e+08
4,Multi-Family with 2 - 4 Units,446,5.939474e+06,1.385683e+08


In [12]:
baseline_agg_query2 = my_run.agg.aggregate_annual(enduses=['fuel_use_natural_gas_total_m_btu'],
                                       group_by=['geometry_building_type_recs'],
                                          get_query_only=True)
print(baseline_agg_query2)

SELECT res_test_03_2018_10k_20220607_baseline."build_existing_model.geometry_building_type_recs" AS geometry_building_type_recs, sum(1) AS sample_count, sum(res_test_03_2018_10k_20220607_baseline."build_existing_model.sample_weight") AS units_count, sum(res_test_03_2018_10k_20220607_baseline."report_simulation_output.fuel_use_natural_gas_total_m_btu" * res_test_03_2018_10k_20220607_baseline."build_existing_model.sample_weight") AS fuel_use_natural_gas_total_m_btu 
FROM res_test_03_2018_10k_20220607_baseline 
WHERE res_test_03_2018_10k_20220607_baseline.completed_status = 'Success' GROUP BY 1 ORDER BY 1


In [13]:
bq = my_run.submit_batch_query([baseline_agg_query, baseline_agg_query2])
print(bq)
my_run.get_batch_query_report(bq)

1


{'submitted': 0, 'running': 0, 'pending': 2, 'completed': 0, 'failed': 0}

In [14]:
my_run.get_batch_query_report(bq)

{'submitted': 0, 'running': 0, 'pending': 2, 'completed': 0, 'failed': 0}

In [15]:
result_df = my_run.get_batch_query_result(bq, combine=True)
result_df

INFO:buildstock_query.query_core:{'submitted': 0, 'running': 0, 'pending': 2, 'completed': 0, 'failed': 0}
INFO:buildstock_query.query_core:{'submitted': 0, 'running': 0, 'pending': 2, 'completed': 0, 'failed': 0}
INFO:buildstock_query.query_core:Submitted queries[0] (e86299b8-442f-45c8-a132-d9fb2ee37b53)
INFO:buildstock_query.query_core:Submitted queries[1] (629084ff-5c02-43f6-8694-ed805e5024fc)
INFO:buildstock_query.query_core:{'submitted': 2, 'running': 0, 'pending': 0, 'completed': 2, 'failed': 0}
INFO:buildstock_query.query_core:Batch query completed. 
INFO:buildstock_query.query_core:Got result from Query [0] (e86299b8-442f-45c8-a132-d9fb2ee37b53)
INFO:buildstock_query.query_core:Got result from Query [1] (629084ff-5c02-43f6-8694-ed805e5024fc)
INFO:buildstock_query.query_core:Concatenating the results.


,geometry_building_type_recs,sample_count,units_count,fuel_use_electricity_total_m_btu,query_id,fuel_use_natural_gas_total_m_btu
0,Mobile Home,703,9.361996e+06,4.506122e+08,0,NaN
1,Multi-Family with 2 - 4 Units,446,5.939474e+06,1.385683e+08,0,NaN
2,Multi-Family with 5+ Units,1189,1.583416e+07,4.099084e+08,0,NaN
3,Single-Family Attached,530,7.058119e+06,2.094629e+08,0,NaN
4,Single-Family Detached,7121,9.483182e+07,4.413546e+09,0,NaN
0,Mobile Home,703,9.361996e+06,NaN,1,1.467286e+08
1,Multi-Family with 2 - 4 Units,446,5.939474e+06,NaN,1,3.638842e+08
2,Multi-Family with 5+ Units,1189,1.583416e+07,NaN,1,2.137026e+08
3,Single-Family Attached,530,7.058119e+06,NaN,1,3.356744e+08
4,Single-Family Detached,7121,9.483182e+07,NaN,1,6.115350e+09


In [16]:
query1 = my_run.agg.aggregate_annual(enduses=['fuel_use_natural_gas_total_m_btu'], group_by=['state'],
                                              get_query_only=True)
query2 = my_run.agg.aggregate_annual(enduses=['fuel_use_electricity_total_m_btu'],
                                              group_by=['state'],
                                              get_query_only=True)
exe_id1, future1 = my_run.execute(query1, run_async=True)
exe_id2, future2 = my_run.execute(query2, run_async=True)

assert exe_id1 != "CACHED"  # Commenting this line will cause the type checker to complain on the next line
my_run.get_query_output_location(exe_id1)

's3://eulp/athena_query_results/6e3daed0-a652-47f5-8e61-5d919b46740a.csv'

In [17]:
future1.as_pandas()

,state,sample_count,units_count,fuel_use_natural_gas_total_m_btu
0,AL,173,2.303877e+06,2.746831e+07
1,AR,100,1.331721e+06,3.919417e+07
2,AZ,217,2.889834e+06,3.297230e+07
3,CA,1051,1.399638e+07,4.360101e+08
4,CO,171,2.277242e+06,1.805907e+08
5,CT,115,1.531479e+06,5.755829e+07
6,DC,22,2.929785e+05,1.760066e+07
7,DE,32,4.261506e+05,2.158285e+07
8,FL,689,9.175555e+06,1.611552e+07
9,GA,308,4.101699e+06,1.151046e+08


In [18]:
my_run.get_query_status(ExeId(exe_id1))

'SUCCEEDED'

In [19]:
# my_run.get_athena_query_result(exe_id1)


In [20]:
my_run.get_all_running_queries()

[]

In [21]:
my_run.stop_all_queries()

INFO:buildstock_query.query_core:Stopped 0 queries


In [22]:
my_run.stop_query(ExeId(exe_id1))

{'ResponseMetadata': {'RequestId': '3f6d2809-1ff7-46ba-a455-dd3cdfbc54fb',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Fri, 14 Apr 2023 15:56:16 GMT',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '2',
   'connection': 'keep-alive',
   'x-amzn-requestid': '3f6d2809-1ff7-46ba-a455-dd3cdfbc54fb'},
  'RetryAttempts': 0}}

In [23]:
my_run.delete_everything()

Will delete the following tables ['res_test_03_2018_10k_20220607_baseline', 'res_test_03_2018_10k_20220607_timeseries', 'res_test_03_2018_10k_20220607_upgrades'] and the s3:/euss/tests/res_test_03_2018_10k_20220607 folder
Abandoned the idea.


In [6]:
# Mix and match: baseline from a different run from upgrades
my_run2 = BuildStockQuery(db_name='largeee_test_runs',
                          workgroup='largeee',
                          table_name=(baseline_20230525_baseline,cateory_2_20230525_timeseries,cateory_2_20230525_upgrades"),
                          buildstock_type='resstock',
                          skip_reports=False)


INFO:buildstock_query.query_core:Loading ('baseline_20230525_baseline', 'cateory_2_20230525_timeseries', 'cateory_2_20230525_upgrades') ...
INFO:buildstock_query.main:Getting Success counts...
INFO:buildstock_query.report_query:Checking integrity with ts_tables ...


         Fail  Unapplicaple  Success     Sum  Applied %  no-chng  bad-chng  \
upgrade                                                                      
0           7             0     9931  9938.0        0.0        0         0   
1           0             0     9931  9931.0      100.0        4       149   
2           0             0     9931  9931.0      100.0        3       391   
3           0          1793     8138  9931.0       81.9        4        16   
4           0           322     9609  9931.0       96.8      123        17   
5           0             0     9931  9931.0      100.0        2       222   

         ok-chng  true-bad-chng  true-ok-chng  null   any  no-chng %  \
upgrade                                                                
0              0              0             0     0     0        0.0   
1           9778            112          9815     0  9931        0.0   
2           9537            292          9637     0  9931        0.0   
3           811

INFO:buildstock_query.query_core:12 queries cache saved to ('baseline_20230525_baseline', 'cateory_2_20230525_timeseries', 'cateory_2_20230525_upgrades')_query_cache.pkl


All buildings are verified to have the same number of (8760) timeseries rows.
